In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/oniondata/weekly_wholesale_price_onion-upto_2012_1.csv
/kaggle/input/potatodataset/weekly_wholesale_price_potato-upto_2012.csv
/kaggle/input/ricedataset/weekly_wholesale_price_rice-upto_2012.csv
/kaggle/input/weekly-year-wise/weekly_wholesale_price_wheat-upto_2012.csv
/kaggle/input/milkdata/weekly_wholesale_price_milk-upto_2012.csv


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# import pandas as pd

# # Function to process and save the top k locations for a given crop
# def process_top_k_locations_per_crop(df, crop_name, k=5):
#     """
#     Process the top k locations in the DataFrame for a given crop based on the number of non-missing values,
#     and capitalize 'chandigarh' to 'CHANDIGARH' in the 'centre' column.
    
#     Parameters:
#     df (DataFrame): The DataFrame containing the data for one crop.
#     crop_name (str): The name of the crop (used for file naming).
#     k (int): The number of top locations to return based on non-missing values.

#     Returns:
#     None
#     """
#     # Capitalize 'chandigarh' to 'CHANDIGARH'
#     df.loc[df['centre'] == 'chandigarh', 'centre'] = 'CHANDIGARH'

#     # Collect locations with their count of non-missing values
#     location_non_missing = []

#     # Calculate non-missing values for each location
#     for location in df['centre'].unique():
#         non_missing_count = df[df['centre'] == location].notna().sum().sum()
#         location_non_missing.append((location, non_missing_count))

#     # Sort locations based on the number of non-missing values in descending order
#     location_non_missing.sort(key=lambda x: x[1], reverse=True)

#     # Select the top k locations
#     top_k_locations = location_non_missing[:k]

#     # Save the filtered data for the top k locations to CSV files
#     for location, _ in top_k_locations:
#         filtered = df[df['centre'] == location].dropna()
#         filtered = filtered[['Date', 'centre', 'Price']]
#         filtered.to_csv(f'{crop_name}.csv', index=False)

# # Load datasets
# wheat_df = pd.read_csv('/kaggle/input/weekly-year-wise/weekly_wholesale_price_wheat-upto_2012.csv')
# rice_df = pd.read_csv('/kaggle/input/ricedataset/weekly_wholesale_price_rice-upto_2012.csv')
# onion_df = pd.read_csv('/kaggle/input/oniondata/weekly_wholesale_price_onion-upto_2012_1.csv')
# potato_df = pd.read_csv('/kaggle/input/potatodataset/weekly_wholesale_price_potato-upto_2012.csv')

# # Process and save data for the top 5 locations for each crop
# process_top_k_locations_per_crop(wheat_df, "wheat", k=5)
# process_top_k_locations_per_crop(rice_df, "rice", k=5)
# process_top_k_locations_per_crop(onion_df, "onion", k=5)
# process_top_k_locations_per_crop(potato_df, "potato", k=5)
# #

In [2]:
import pandas as pd

def process_and_combine_top_k_locations_per_crop(crop_dfs, k=5):
    """
    Process the top k locations for each crop based on the longest continuous time series,
    combine the data for all locations for each crop, and save to CSV files.
    
    Parameters:
    crop_dfs (dict): A dictionary with crop names as keys and DataFrames as values.
    k (int): The number of top locations to process for each crop.
    
    Returns:
    None
    """
    for crop_name, df in crop_dfs.items():
        # Convert 'Date' to datetime
        df['Date'] = pd.to_datetime(df['Date'])
        
        # Capitalize 'chandigarh' to 'CHANDIGARH'
        df.loc[df['centre'] == 'chandigarh', 'centre'] = 'CHANDIGARH'
        
        # Calculate the longest continuous time series for each location
        location_time_series = []
        for location in df['centre'].unique():
            location_df = df[df['centre'] == location].sort_values('Date')
            if not location_df.empty:
                date_diffs = location_df['Date'].diff().dt.days
                max_continuous = (date_diffs != 7).cumsum().value_counts().max()
                location_time_series.append((location, max_continuous))
        
        # Sort locations based on the length of continuous time series in descending order
        location_time_series.sort(key=lambda x: x[1], reverse=True)
        
        # Select the top k locations
        top_k_locations = [loc for loc, _ in location_time_series[:k]]
        
        # Filter data for top k locations and combine
        combined_data = []
        for location in top_k_locations:
            filtered = df[df['centre'] == location].sort_values('Date')
            filtered = filtered[['Date', 'centre', 'Price']]
            combined_data.append(filtered)
        
        # Combine data for all top locations for this crop
        if combined_data:
            combined_df = pd.concat(combined_data, ignore_index=True)
            combined_df = combined_df[['Date', 'centre', 'Price']]
            
            # Sort the combined data by Date
            combined_df = combined_df.sort_values('Date')
            combine_df = combined_df.dropna()
            # Save to CSV
            combined_df.to_csv(f'{crop_name}_top_{k}_locations_timeseries.csv', index=False)

# Load datasets
crop_dfs = {
    'wheat': pd.read_csv('/kaggle/input/weekly-year-wise/weekly_wholesale_price_wheat-upto_2012.csv'),
    'rice': pd.read_csv('/kaggle/input/ricedataset/weekly_wholesale_price_rice-upto_2012.csv'),
    'onion': pd.read_csv('/kaggle/input/oniondata/weekly_wholesale_price_onion-upto_2012_1.csv'),
    'potato': pd.read_csv('/kaggle/input/potatodataset/weekly_wholesale_price_potato-upto_2012.csv')
}

# Process and save combined data for the top 5 locations for each crop
process_and_combine_top_k_locations_per_crop(crop_dfs, k=5)

/tmp/ipykernel_36/2788684281.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])
/tmp/ipykernel_36/2788684281.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])
/tmp/ipykernel_36/2788684281.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])
/tmp/ipykernel_36/2788684281.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a fo

In [3]:
!zip -r file_combined.zip /kaggle/working

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/rice_top_5_locations_timeseries.csv (deflated 85%)
  adding: kaggle/working/wheat_top_5_locations_timeseries.csv (deflated 85%)
  adding: kaggle/working/onion_top_5_locations_timeseries.csv (deflated 85%)
  adding: kaggle/working/potato_top_5_locations_timeseries.csv (deflated 85%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)


In [4]:
from IPython.display import FileLink
FileLink(r'file_combined.zip')

/kaggle/working/file_combined.zip

In [13]:
import pandas as pd

def process_and_combine_top_k_locations_per_crop(crop_dfs, k=5):
    """
    Process the top k locations for each crop based on the longest continuous time series,
    combine the data for all locations for each crop, and save to CSV files.
    
    Parameters:
    crop_dfs (dict): A dictionary with crop names as keys and DataFrames as values.
    k (int): The number of top locations to process for each crop.
    
    Returns:
    None
    """
    for crop_name, df in crop_dfs.items():
        # Convert 'Date' to datetime
        df['Date'] = pd.to_datetime(df['Date'])
        
        # Capitalize 'chandigarh' to 'CHANDIGARH'
        df.loc[df['centre'] == 'chandigarh', 'centre'] = 'CHANDIGARH'
        
        # Calculate the longest continuous time series for each location
        location_time_series = []
        for location in df['centre'].unique():
            location_df = df[df['centre'] == location].sort_values('Date')
            if not location_df.empty:
                date_diffs = location_df['Date'].diff().dt.days
                max_continuous = (date_diffs != 7).cumsum().value_counts().max()
                location_time_series.append((location, max_continuous))
        
        # Sort locations based on the length of continuous time series in descending order
        location_time_series.sort(key=lambda x: x[1], reverse=True)
        
        # Select the top k locations
        top_k_locations = [loc for loc, _ in location_time_series[:k]]
        
        # Filter data for top k locations and combine
        combined_data = []
        for location in top_k_locations:
            filtered = df[df['centre'] == location].sort_values('Date')
            filtered = filtered[['Date', 'centre', 'Price']]
            combined_data.append(filtered)
        
        # Combine data for all top locations for this crop
        if combined_data:
            combined_df = pd.concat(combined_data, ignore_index=True)
            combined_df = combined_df[['Date', 'centre', 'Price']]
            
            # Sort the combined data by Date
            combined_df = combined_df.sort_values('Date')
            combine_df = combined_df.dropna()
            # Save to CSV
            combined_df.to_csv(f'{crop_name}_top_{k}_locations_timeseries.csv', index=False)

# Load datasets
crop_dfs = {
    'wheat': pd.read_csv('/kaggle/input/weekly-year-wise/weekly_wholesale_price_wheat-upto_2012.csv'),
    'rice': pd.read_csv('/kaggle/input/ricedataset/weekly_wholesale_price_rice-upto_2012.csv'),
    'onion': pd.read_csv('/kaggle/input/oniondata/weekly_wholesale_price_onion-upto_2012_1.csv'),
    'potato': pd.read_csv('/kaggle/input/potatodataset/weekly_wholesale_price_potato-upto_2012.csv')
}

# Process and save combined data for the top 5 locations for each crop
process_and_combine_top_k_locations_per_crop(crop_dfs, k=5)

/tmp/ipykernel_36/1679371191.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


Date      0
centre    0
Price     0
dtype: int64